In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import tensorflow as tf
import os
import numpy as np
from prosit_t.models import PrositTransformerV2
from dlomix.losses import masked_spectral_distance

2023-09-20 13:34:25.334801: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 13:34:26.069290: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
tf.config.run_functions_eagerly(True)

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [57]:
sequences_data = [
    ["S","E","Q"],
    ["S","E"],
    ["S","E","Q", "W"],
    ["E", "Q"],
    ["S","E","Q", "W", "E", "N", "C", "E"],
    ["S","E","Q", "W", "E", "N", "C", "E", "T", "Y"],
]

labels_data = []
for i in sequences_data:
    labels_data.append(list(range(len(i)-1))*6)

charge_data = [
    [0,1,0,0,0,0],
    [0,0,1,0,0,0],
    [0,0,1,0,0,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,1],
    [0,1,0,0,0,0]
]
ce_data = [
    [0.215],
    [0.24],
    [0.287],
    [0.213],
    [0.215],
    [0.234]
]
sequences_ragged = tf.ragged.constant(sequences_data, dtype=tf.string)
labels_ragged = tf.ragged.constant(labels_data, dtype=tf.float64)
charge_ragged = tf.ragged.constant(charge_data, dtype=tf.float32)
ce_ragged = tf.ragged.constant(ce_data, dtype=tf.float64)

In [58]:
def ragged_to_dense(x,y):
    return x, y

def merge_tuples(item1, item2):
    return ({
        "sequence": item1[0],
        "collision_energy": item2[1],
        "precursor_charge": item2[0],
    }, item1[-1])

In [59]:
dataset = tf.data.Dataset.from_tensor_slices(
            (
                sequences_ragged,
                labels_ragged,
            )
        ).map(ragged_to_dense).padded_batch(2).unbatch()

In [60]:
dataset_meta = tf.data.Dataset.from_tensor_slices(
    (
        charge_ragged,
        ce_ragged
    )).map(ragged_to_dense)

In [61]:
full = tf.data.Dataset.zip(dataset, dataset_meta).map(merge_tuples).batch(2).prefetch(tf.data.AUTOTUNE)

In [118]:
from prosit_t.models.test_model_decoder import TestModelDecoder
from prosit_t.models.test_model import TestModel
from prosit_t.models.test_model_drop import TestModelDrop
from prosit_t.models.test_model_pooling import TestModelPooling
import keras.backend as K

In [119]:
test = TestModelDecoder(embedding_output_dim=64,num_heads=16,num_transformers=3,dense_dim_factor=4)

ValueError: Dimension -1 must be >= 0

In [115]:
test.compile(loss=masked_spectral_distance, optimizer='adam')

In [116]:
test.fit(full, epochs=10)

Epoch 1/10
(2, 3, 64)
(2, 3, 6)
(2, 2, 6)
      1/Unknown - 0s 319ms/step - loss: 0.9915(2, 4, 64)
(2, 3, 6)
(2, 2, 6)


2023-09-21 11:53:10.968987: W tensorflow/core/framework/op_kernel.cc:1816] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: in user code:

    File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/dlomix/losses/intensity.py", line 18, in masked_spectral_distance  *
        pred_masked = ((y_true + 1) * y_pred) / (y_true + 1 + epsilon)

    InvalidArgumentError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} required broadcastable shapes [Op:Mul] name: 


In [103]:
import plotly.express as px

fig = px.line(x=list(range(1,11)), y=test.history.history["loss"], title='TimeDistributed Dense Layer')
fig.update_xaxes(title='Epoch')
fig.update_yaxes(title='Loss')

In [104]:
test.summary()

Model: "test_model_decoder_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_34 (Embedding)    multiple                  0 (unused)
                                                                 
 flatten_17 (Flatten)        multiple                  0         
                                                                 
 time_distributed_17 (TimeD  multiple                  390       
 istributed)                                                     
                                                                 
 string_lookup_17 (StringLo  multiple                  0         
 okup)                                                           
                                                                 
 positional_embedding_17 (P  multiple                  1408      
 ositionalEmbedding)                                             
                                              